### 1. Importing Requirements

Let's import the necessary libraries for our use case. In here there is yaml for configuration management, a machine learning algorithm, a dataframe for data manipulation as well as the artifact generator and deployer.

In [ ]:
from hana_ml.algorithms.pal.unified_classification import UnifiedClassification
import hana_ml.dataframe as dataframe

from hana_ml.artifacts.generators import hana

### 2. Create connection context

In the following code block we just load our credentials from disk in order to not leak them into this notebook or the underlying git repository:

In [ ]:
from hana_ml.algorithms.pal.utility import DataSets, Settings

In [ ]:
try:
    import configparser
except ImportError:
    import ConfigParser as configparser
Settings.settings = configparser.ConfigParser()
Settings.settings.read("../../config/e2edata.ini")
url = Settings.settings.get("hana", "url")
port = Settings.settings.get("hana", "port")
user = Settings.settings.get("hana", "user")
passwd = Settings.settings.get("hana", "passwd")

Now its time to create a connection context for our HANA system. This allows us to access the required data, as well as the PAL procedures we need to call in order to train our model.

In [ ]:
connection_context = dataframe.ConnectionContext(
    url, int(port), user, passwd)


In [ ]:
connection_context.hana_version()

In [ ]:
connection_context.get_current_schema()

We also enable SQL tracing for later reuse of the model in the deployment.

### 4. Prepare the data

This block is part of the utils for this demo - it makes sure the dataset is in the system and creates it if necessary. In a real production use case this would obviously be unnecessary since the data is already in the system.

In [ ]:

diabetes_full, diabetes_train_valid, diabetes_test, _ = DataSets.load_diabetes_data(connection_context)
diabetes_train_valid = diabetes_train_valid.save("diabetes_train_valid")
diabetes_test = diabetes_test.save("diabetes_test")
Settings.set_log_level()

In [ ]:
connection_context.sql_tracer.enable_sql_trace(True)
connection_context.sql_tracer.enable_trace_history(True)

### 5. Data Science Loop

In this section the real work of a data scientist happens. They manipulate the data, preprocess columns, choose a model and try different combinations of hyper parameters.

Since we just want to demonstrate the deployment, lets keep this short and just use a basic Random Decision Tree Classifier.

In [ ]:

rfc_params = dict(n_estimators=5, split_threshold=0, max_depth=10)
rfc = UnifiedClassification(func="RandomDecisionTree", **rfc_params)
rfc.fit(diabetes_train_valid, 
        key='ID', 
        label='CLASS', 
        categorical_variable=['CLASS'],
        partition_method='stratified',
        stratified_column='CLASS',)
cm = rfc.confusion_matrix_.collect()
rfc.predict(diabetes_test.drop(cols=['CLASS']), key="ID")

We can also view the confusion matrix and accuracy:

In [ ]:
print(cm)
print(float(cm['COUNT'][cm['ACTUAL_CLASS'] == cm['PREDICTED_CLASS']].sum()) / cm['COUNT'].sum())

### 4. Generate HDI artifact


In [ ]:
hg = hana.HanaGenerator(project_name="test", version='1', grant_service='', connection_context=connection_context, outputdir="./hana_out")

In [ ]:
hg.config.config

In [ ]:
hg.generate_artifacts()